In [2]:
##########----- Reused code -----##########
# Last update: Sept 13, 2020
# By: Huitian Diao
###########################################

library(stringr)
library(cowplot)
library(ggpubr)

code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-1)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/plotting_codes", "", code_dir)
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC"

## All louvain clusters - GSEA summary plots

In [7]:
exp_name <- "0_Acute-Chronic"
analysis_name <- c("0", "resampled") 

In [8]:
new_names <- file.path(base_dir,'/0_Acute-Chronic/2_scVelo/0_scVelo_out_resampledNoDyn/pt_ordered_cluster_avg.csv')
new_names_df <- read_csv(new_names) %>% select(one_of('louvain', 'louvain_new_label'))
colnames(new_names_df) <- c('comparison', 'louvain_new_label')
cp_order <- new_names_df$louvain_new_label
cp_order <- paste(rep(cp_order, each=2), rep(c("A","C"), length(cp_order)), sep="_")
cp_order

Parsed with column specification:
cols(
  louvain = col_double(),
  initial_size_spliced = col_double(),
  initial_size_unspliced = col_double(),
  initial_size = col_double(),
  n_counts = col_double(),
  dpt_pseudotime = col_double(),
  velocity_self_transition = col_double(),
  root_cells = col_double(),
  end_points = col_double(),
  velocity_pseudotime = col_double(),
  louvain_new_label = col_character()
)



[1] "P0_A"  "P0_C"  "P1_A"  "P1_C"  "P2_A"  "P2_C"  "P3_A"  "P3_C"  "P4_A" 
[10] "P4_C"  "P5_A"  "P5_C"  "P6_A"  "P6_C"  "P7_A"  "P7_C"  "P8_A"  "P8_C" 
[19] "P9_A"  "P9_C"  "P10_A" "P10_C"

In [9]:
###----- Find all GSEA outputs for Louvain clusters
sum.dir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic/1_Scanpy/0_Scanpy_out_resampled/3_GSEA'
use.key <- 'louvain_celltype_general_withNAV'
use.path <- file.path(base_dir, '9_Figures', exp_name, 
                      paste(analysis_name, collapse="_"),
                      paste(use.key, 'sum', 'plot', sep="_"))
dir.create(use.path, showWarnings = FALSE, recursive=TRUE)
setwd(use.path)

input.path <- sum.dir
input.files <- list.files(input.path, pattern='.csv', full.name=TRUE)
input.files <- input.files[grepl(paste(use.key, 'gsea', sep="_"), input.files)]

out.name <- paste(use.key, 'gsea', 'all.csv', sep="_")

###----- Combine all leiden cluster GSEA outputs
df_all <- read_csv(input.files[1])
for (file in input.files[2:length(input.files)]){
    df_all <- df_all %>% bind_rows(read_csv(file))
}

df_all <- df_all %>% separate(comparison, c("comparison", "celltype"))
df_all$comparison <- as.numeric(df_all$comparison)

###----- Rename
df_all <- df_all %>% left_join(new_names_df, by="comparison")
df_all$old_names <- paste(df_all$comparison, df_all$celltype, sep="_")
df_all$comparison <- paste(df_all$louvain_new_label, df_all$celltype, sep="_")
df_all$louvain_new_label <- NULL
df_all$celltype <- NULL

write_csv(df_all, out.name)

Parsed with column specification:
cols(
  comparison = col_character(),
  pathway = col_character(),
  NES = col_double(),
  padj = col_double(),
  leadingEdge_signal = col_double(),
  mlog10padj = col_double()
)

Parsed with column specification:
cols(
  comparison = col_character(),
  pathway = col_character(),
  NES = col_double(),
  padj = col_double(),
  leadingEdge_signal = col_double(),
  mlog10padj = col_double()
)

Parsed with column specification:
cols(
  comparison = col_character(),
  pathway = col_character(),
  NES = col_double(),
  padj = col_double(),
  leadingEdge_signal = col_double(),
  mlog10padj = col_double()
)

Parsed with column specification:
cols(
  comparison = col_character(),
  pathway = col_character(),
  NES = col_double(),
  padj = col_double(),
  leadingEdge_signal = col_double(),
  mlog10padj = col_double()
)

Parsed with column specification:
cols(
  comparison = col_character(),
  pathway = col_character(),
  NES = col_double(),
  padj = col_double()

In [10]:
plot.use.gs <- '/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/Y_annotated/anno_plotuse_gs_20200928.csv'
plot.use.gs.df <- read_csv(plot.use.gs)

use.types <- unique(plot.use.gs.df$plot_use)
use.types <- use.types[!is.na(use.types)]
print(use.types)

Parsed with column specification:
cols(
  gs_name = col_character(),
  type = col_character(),
  use = col_character(),
  plot_use = col_character(),
  abbr = col_character(),
  order = col_double()
)



[1] "Effector" "Tex"      "MEM"      "Best"    


In [ ]:
###----- For each subset of Gene signatures, plot heatmap

for (use_type in use.types) {
    print(use_type)
    use_type_gs_df <- plot.use.gs.df %>% filter(plot_use == use_type) %>% select(one_of(c("gs_name", "abbr", 'order')))
    colnames(use_type_gs_df) <- c("pathway", "abbr", 'order')
    

    df_plot <- df_all %>% filter(pathway %in% use_type_gs_df$pathway) %>% left_join(use_type_gs_df, by='pathway')
    df_plot$comparison <- factor(df_plot$comparison, levels=cp_order)
    padding <- 25 - max(nchar(as.character(df_plot$abbr)))
    padding <- paste(rep(" ", padding), collapse="")
    df_plot$abbr <- paste(padding, df_plot$abbr)
    use_type_gs_df$abbr <- paste(padding, use_type_gs_df$abbr)
    gs_abbr_order <- use_type_gs_df %>% arrange(order) %>% .$abbr
    df_plot$abbr <- factor(df_plot$abbr, levels=gs_abbr_order)
    
    #--- Plotting
    
    font_size = 10
    bbplot <- ggplot(df_plot, aes(comparison, abbr)) +
        geom_point(aes(size=mlog10padj, color=NES)) +
        scale_color_gradient2(low="blue", mid="white", high="red", midpoint=0, limits=c(-4,4)) +
        scale_size(range=c(0,4), breaks = c(0,1,2,3),labels=c('0','1','2', '>=3')) +
        theme(axis.title.x=element_blank(), 
              axis.title.y=element_blank(), 
              text=element_text(size=font_size, family='mono'),
              axis.text.x = element_text(angle = 45, hjust=1, vjust=1))
    leg <- get_legend(bbplot)
    bbplot_leg <- as_ggplot(leg)
    
    bbplot <- bbplot + theme(legend.position='none')
    
    #--- Save output plot
    wid = length(unique(df_plot$comparison)) * 0.9
    hei = length(unique(df_plot$abbr)) * 0.4 + 0.2*font_size
    
    outname_pdf = paste(use.key, "_",use_type, ".pdf", sep="")
    outname_pdf_leg = paste(use.key, "_",use_type, "_legend.pdf", sep="")
    ggsave(outname_pdf, bbplot, device="pdf", width=wid, height=hei, units="cm")
    ggsave(outname_pdf_leg, bbplot_leg, device="pdf", width=5, height=10, units="cm")
    
    outname_pdf = paste(use.key, "_",use_type, ".png", sep="")
    outname_pdf_leg = paste(use.key, "_",use_type, "_legend.png", sep="")    
    ggsave(outname_pdf, bbplot, device="png", width=wid, height=hei, units="cm")
    ggsave(outname_pdf_leg, bbplot_leg, device="png", width=5, height=10, units="cm")
}

[1] "Effector"
[1] "Tex"
[1] "MEM"
[1] "Best"
